# Benches NTA

Trees per unit length of street.


In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing


%matplotlib inline

In [2]:
from shapely.geometry import LineString, Polygon, Point, LinearRing
from shapely.ops import transform
from functools import partial
import pyproj

## Benches 

In [3]:
benchescount = pd.read_csv('../../data/streets/benchesNTA.csv')
plazscount = pd.read_csv('../../data/streets/plazasNTA.csv')
streetlen = pd.read_csv('../../data/streets/StreetLenNTA.csv')

### Data Reading

In [5]:
plazscount.head(2)

,FID,BoroCode,CountyFIPS,NTACode,NTAName,Count_plazas
0,70,4,81,QN30,Maspeth,11
1,170,1,61,MN24,SoHo-TriBeCa-Civic Center-Little Italy,9


In [7]:
# creating files with only bench count and NTA & Plazas count and NTA
benachc = benchescount[['NTACode','Count_Benches']]
plazasc = plazscount[['NTACode','Count_plazas']]


In [8]:
#merging on NTA
benachc = benachc.merge(plazasc, on='NTACode', how='inner')


In [9]:
benachc.head(5)

,NTACode,Count_Benches,Count_plazas
0,BK88,10,3
1,QN52,2,2
2,QN48,2,0
3,QN51,1,3
4,QN27,2,4


In [10]:
# count = aggragatiion benches per NTA for plazas + benches 
benachc['count'] = benachc['Count_Benches'] + benachc['Count_plazas']

In [11]:
# creating a new DF. NTAbenchplaz is the inclusive source for number of bencehs and plazas per NTA 
NTAbenchplaz = benachc[['NTACode','count']]


### Merging Length and Number of Benches per NTA

In [36]:
NTAbenchplaz['NTA'] = NTAbenchplaz['NTACode']


/opt/rh/anaconda/root/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [37]:
streetlen = streetlen.merge(NTAbenchplaz ,on='NTA', how='inner')



In [38]:
streetlen.head(10)

,Unnamed: 0,NTA_STREET_LEN,NTA,NTACode,count
0,1,84074.400771,BK09,BK09,18
1,2,416142.744684,BK17,BK17,7
2,3,155225.173873,BK19,BK19,2
3,4,164313.426249,BK21,BK21,10
4,5,43969.882557,BK23,BK23,1
5,6,183791.636138,BK25,BK25,5
6,7,145410.117910,BK26,BK26,4
7,8,137352.828666,BK27,BK27,2
8,9,270872.626524,BK28,BK28,11
9,10,221205.290930,BK29,BK29,7


In [39]:


streetlen['BENCHES_PER_UNIT_LENGTH'] = streetlen['count'] / streetlen['NTA_STREET_LEN']  


In [40]:
std_scaler = preprocessing.StandardScaler()
streetlen['STD_benches_PER_UNIT_LENGTH'] = std_scaler.fit_transform(streetlen['BENCHES_PER_UNIT_LENGTH'])


In [41]:
streetlen.head(2)

,Unnamed: 0,NTA_STREET_LEN,NTA,NTACode,count,BENCHES_PER_UNIT_LENGTH,STD_benches_PER_UNIT_LENGTH
0,1,84074.400771,BK09,BK09,18,0.000214,3.468791
1,2,416142.744684,BK17,BK17,7,0.000017,-0.669869


In [42]:
benches_quality_indicator = streetlen[['NTACode','STD_benches_PER_UNIT_LENGTH']]

In [43]:
benches_quality_indicator.to_csv('StreetBenchesIndicator.csv')

In [44]:
benches_quality_indicator

,NTACode,STD_benches_PER_UNIT_LENGTH
0,BK09,3.468791
1,BK17,-0.669869
2,BK19,-0.752457
3,BK21,0.254014
4,BK23,-0.545637
5,BK25,-0.452031
6,BK26,-0.445660
7,BK27,-0.717284
8,BK28,-0.170810
9,BK29,-0.358881


In [45]:
streetlen

,Unnamed: 0,NTA_STREET_LEN,NTA,NTACode,count,BENCHES_PER_UNIT_LENGTH,STD_benches_PER_UNIT_LENGTH
0,1,84074.400771,BK09,BK09,18,0.000214,3.468791
1,2,416142.744684,BK17,BK17,7,0.000017,-0.669869
2,3,155225.173873,BK19,BK19,2,0.000013,-0.752457
3,4,164313.426249,BK21,BK21,10,0.000061,0.254014
4,5,43969.882557,BK23,BK23,1,0.000023,-0.545637
5,6,183791.636138,BK25,BK25,5,0.000027,-0.452031
6,7,145410.117910,BK26,BK26,4,0.000028,-0.445660
7,8,137352.828666,BK27,BK27,2,0.000015,-0.717284
8,9,270872.626524,BK28,BK28,11,0.000041,-0.170810
9,10,221205.290930,BK29,BK29,7,0.000032,-0.358881
